<a href="https://colab.research.google.com/github/szambetti/INVEST_IO/blob/master/stock_scraping_investing_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Questo notebook scarica i vari calendari da investing, le date sono settabili come riportato sotto



In [ ]:
#mounts current gdrive in colab after providing success code
#INSERISCI IL CODICE DI SUCCESSO O NON FUNZIONERA'
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/INVEST-IO/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/INVEST-IO


In [ ]:
#apply filter on dates, scarica sempre dallo startdate (da indicare sotto) a oggi, !!! FORMATO DATE US !!!
startdate = '01/01/2020'

In [ ]:
%%capture
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as progress_bar
import pandas
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


CALENDARIO ECONOMICO

In [ ]:
#importa la data di oggi nel formato corretto
from datetime import date
import time
today = date.today()
today_us_format = today.strftime("%m/%d/%Y")

In [ ]:
#function per cliccare pulsanti su chrome .click() di selenium non funziona
def click_on_button_id(x):
  element = wd.find_element_by_id(x)
  wd.execute_script("arguments[0].click();", element)

In [ ]:
#CALENDARIO ECONOMICO

url = 'https://www.investing.com/economic-calendar'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 5

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height


list_of_news = []

#launch main scraping loop
tr_list = wd.find_elements_by_xpath('//*[@id="economicCalendarData"]/tbody//tr')
for tr in tr_list:
    td_list = tr.find_elements_by_tag_name('td')
    list_of_news_data=[]
    for td in td_list:
      if td.text != '':
        list_of_news_data.append(td.text)
      elif td.get_attribute('title') != '':
        list_of_news_data.append(td.get_attribute('title'))
      else:
        list_of_news_data.append('')
    if len(list_of_news_data) == 1:
        date = list_of_news_data[0]
    elif len(list_of_news_data) == 4:
      list_of_news.append({'date' : date, 'time': list_of_news_data[0], 
                            'currency' : list_of_news_data[1],
                            'importance' : list_of_news_data[2], 
                            'event' : list_of_news_data[3],
                            'actual' : '',
                            'expected' : '',
                            'previous' : ''})
    else:
      list_of_news.append({'date' : date, 'time': list_of_news_data[0], 
                            'currency' : list_of_news_data[1],
                            'importance' : list_of_news_data[2], 
                            'event' : list_of_news_data[3],
                            'actual' : list_of_news_data[4],
                            'expected' : list_of_news_data[5],
                            'previous' : list_of_news_data[6]})

In [ ]:
from pandas import DataFrame
df = DataFrame (list_of_news)
df.head()
df.to_csv('sr_economic_calendar.csv')

In [ ]:
df.head(15)

CALENDARIO EARNINGS

In [ ]:
#CALENDARIO EARNINGS

url = 'https://www.investing.com/earnings-calendar/'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 5

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

output_list = []

#main scraper loop
tr_list = wd.find_elements_by_xpath('//*[@id="earningsCalendarData"]/tbody//tr')
for tr in progress(tr_list):
  td_list = tr.find_elements_by_tag_name('td')
  temp_list = []
  for td in td_list:
    if td.text != '':
      temp_list.append(td.text)
    elif td.get_attribute('title') != '':
      temp_list.append(td.get_attribute('title'))
    else:
      temp_list.append('')
  if len(temp_list) > 1:
    output_list.append({'date' : date, 'company': temp_list[1], 
                          'EPS' : temp_list[2],
                          'EPS Forecast' : temp_list[3], 
                          'Revenue' : temp_list[4],
                          'Revenue Forecast' : temp_list[5],
                          'Market Cap' : temp_list[6]})

In [ ]:
from pandas import DataFrame
df = DataFrame (output_list)
df.to_csv('sr_earnings_calendar.csv')

In [ ]:
df.head(15)

CALENDARIO DIVIDENDS

In [ ]:
#CALENDARIO DIVIDENDS

url = 'https://www.investing.com/dividends-calendar/'

wd.set_window_size(1920,1080)
wd.maximize_window()
wd.get(url)

#clicca sui pulsanti per filtrare la data
click_on_button_id('datePickerToggleBtn')
click_on_button_id('startDate')
wd.find_element_by_id("startDate").clear()
wd.find_element_by_id("startDate").send_keys(startdate)
click_on_button_id('endDate')
wd.find_element_by_id("endDate").clear()
wd.find_element_by_id("endDate").send_keys(today_us_format)
click_on_button_id("applyBtn")

#wait for the apply button to be effective
time.sleep(7)


#scrolla fino alla fine della pagina, aspetta questo timeout per caricare i dati
SCROLL_PAUSE_TIME = 5

#scrollare main method
last_height = wd.execute_script("return document.body.scrollHeight")
while True:
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = wd.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

output_list = []

#main scraper loop
tr_list = wd.find_elements_by_xpath('//*[@id="dividendsCalendarData"]/tbody//tr')
for tr in tr_list:
  tds = tr.find_elements_by_tag_name('td')
  temp_list = []
  for td in tds:
    if td.text != '':
      temp_list.append(td.text)
    elif td.get_attribute('data-value') != '':
      temp_list.append(td.get_attribute('data-value'))
    else:
      temp_list.append('')
  if len(temp_list) > 1:
  if temp_list[4] = '1':
    div_type = 'Monthly'
  elif temp_list[4] = '2':
    div_type = 'Quartely'
    elif temp_list[4] = '3':
    div_type = 'Semi-annual'
    elif temp_list[4] = '4':
    div_type = 'Annual'
    elif temp_list[4] = '5':
    div_type = 'Trailing Twelve Months'
    output_list.append({'company': temp_list[1], 
                          'Ex-dividend date' : temp_list[2],
                          'Dividend' : temp_list[3], 
                          'Type' : temp_list[4],
                          'Payment date' : temp_list[5],
                          'Yield' : temp_list[6]})

print(output_list)

In [ ]:
from pandas import DataFrame
df = DataFrame (output_list)
df.to_csv('sr_dividends_calendar.csv')

In [ ]:
df.head(15)